### 1. Import and load packages 

In [ ]:
# Load packages
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
from datetime import date
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery
import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

In [ ]:
pd.set_option("display.max_columns", None)

### 2. Load exposure and outcome cohorts from Big Query 

##### exp_2022 and out_2022

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'

Set up connection

In [ ]:
os.getcwd()

In [ ]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)
#client=bigquery.Client();

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_df = client.query(sql).to_dataframe()

sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
out_df = client.query(sql).to_dataframe()

### 3. Create CLMBR Patient List 

In [ ]:
# Merge outcome and cohort dataframes to create labels column
df = pd.merge(exp_df, out_df, on = 'subject_id', how = 'left')

In [ ]:
df.head()

In [ ]:
df.columns = ['exp_id', 'subject_id', 'exp_cohort_start_date', 'exp_cohort_end_date', 'out_id', 
              'out_cohort_start_date', 'out_cohort_end_date']

In [ ]:
os.getcwd()

In [ ]:
# Bring in patient dataframe with labels for various predictions
patient_df = pd.read_csv('/home/kbechler/patient_df_labels_2022.csv')

In [ ]:
patient_df.head()

In [ ]:
sum(patient_df.ln_5years == 1)

##### Set label for if patient develops LN in the next 5 years 

In [ ]:
# Merge ln_5years with df
df = pd.merge(df, patient_df[['subject_id', 'ln_5years', 'time_sle_to_ln']], on = 'subject_id', how = 'left')

In [ ]:
# Create label column
df['label'] = df.ln_5years

In [ ]:
print("Number of positive cases:", sum(df.label == 1))
print("Number of negative cases:", sum(df.label == 0))
print("Total number of patients in exposure cohort:", len(df))

In [ ]:
df.head()

In [ ]:
# Subset to columns for CLMBR dataframe
df_clmbr = df[['subject_id', 'exp_cohort_start_date', 'label']]

In [ ]:
# Rename columns
df_clmbr.columns = ['patient_id', 'date', 'label']

In [ ]:
# Change to True/False
df_clmbr['label'] = np.where(df_clmbr.label == 1, True, False)

In [ ]:
df_clmbr.head()

### 4. Update outcome cohort in BigQuery to labels with LN within 5 years 

In [ ]:
true_labels = df_clmbr[df_clmbr.label == True].patient_id

In [ ]:
true_labels = tuple(true_labels)

In [ ]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.out_2022` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.out_2022` out_2022
WHERE
subject_id IN{}""".format(true_labels)

client.query(query).result();

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.out_2022"
out_2022 = client.query(sql).to_dataframe()

In [ ]:
print(len(out_2022))

### 5. Check to see if person's birth is post cohort start date

In [ ]:
person_df = pd.read_csv('/home/kbechler/notebooks/data/person.tsv', sep='\t')

In [ ]:
person_df.head()

In [ ]:
len(person_df)

In [ ]:
# Merge with person_df
qc = pd.merge(df_clmbr, person_df[['person_id', 'birth_DATETIME']], left_on = 'patient_id', right_on = 'person_id', 
             how = 'left')

In [ ]:
len(qc)

In [ ]:
qc.head()

In [ ]:
qc.dtypes

In [ ]:
qc.date = pd.to_datetime(qc.date)
qc.birth_DATETIME = pd.to_datetime(qc.birth_DATETIME)

In [ ]:
qc['start_date_birth_dif'] = (qc.date - qc.birth_DATETIME).dt.days

In [ ]:
qc.head()

In [ ]:
qc.start_date_birth_dif.describe()

Confirmed no patient with birth post cohort start date

### 6. Split into train test set

In [ ]:
# Need to split to give Ethan test set of patients to exclude when re-training CLMBR

In [ ]:
df_clmbr.head()

In [ ]:
X = df_clmbr[['patient_id', 'date']]
y = df_clmbr[['label']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 0.8)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
# Import train test split
test_split = pd.read_csv('/home/kbechler/test_set.csv')

### 7. Export clmbr_df to csv 

In [ ]:
os.getcwd()

In [ ]:
# Write to CSV
df_clmbr.to_csv('/home/kbechler/df_clmbr_2022.csv', index = False)

In [ ]:
# Write patients to exclude to CSV
# X_test.to_csv('/home/kbechler/test_set.csv', index = False)